# Head Scatter Calibration

This script models scattered radiation from the treatment head (flattening filter, collimators) using output factor measurements.

**What you need:**
- Output factors for square fields (4×4 to 20×20 cm minimum)
- Output factors for rectangular fields (40×5, 5×40 cm recommended)
- Machine geometry parameters (source-to-detector distances)

**What to do:**
1. Enter your measured output factors in `OF_MEASUREMENTS`
2. Enter field sizes in `FIELD_SIZES` 
3. Verify machine geometry distances (`D_ISO`, `D_FF`, `D_X`, `D_Y`)
4. Run the script - it will fit the head scatter model automatically

**Output:**
- `head_scatter_amplitude` [scatter_fraction, correction_factor]
- `head_scatter_sigma` [sigma_MLC, sigma_Jaw] in cm


In [ ]:
import torch
import numpy as np
from pydose_rt import DoseEngine
from pydose_rt.data import MachineConfig, Phantom, Beam
from pydose_rt.physics.fluence.fluence_modeling import precompute_head_scatter_kernel
from scipy.special import erf
from scipy.optimize import least_squares
import matplotlib.pyplot as plt


In [ ]:
FIELD_SIZES = np.array([[4, 4], [5, 5], [7, 7], [8, 8], [10, 10], [12, 12], [15, 15], [20, 20], [40, 5], [5, 40]])
OF_MEASUREMENTS = np.array([0.884, 0.912, 0.955, 0.971, 1.000, 1.021, 1.045, 1.075, 0.939, 0.963])

# Geometry (cm)
D_ISO = 100.0   # source-to-isocenter distance
D_FF = 10.0     # source-to-flattening-filter distance
D_Y = 25.7      # source-to-Y-jaw distance
D_X = 36.6      # source-to-X-jaw distance


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float32

kernel_size = 201

results = []

machine_config = MachineConfig(
    preset="../../src/pydose_rt/data/machine_presets/umea_10MV.json", 
    head_scatter_amplitude=None,
    output_factors=None,
    head_scatter_sigma=None,
    profile_corrections=None)
resolution = (2.0, 2.0, 2.0)
ct_array_shape = (200, 200, 200)
phantom = Phantom.from_uniform_water(shape=ct_array_shape, spacing=resolution).to(device).to(dtype)
beam_template = Beam.create(
    gantry_angle_deg=0.0, 
    number_of_leaf_pairs=60, 
    collimator_angle_deg=0.0, 
    field_size_mm=(100, 100), 
    iso_center=(200.0, 100.0, 200.0), 
    device=device, 
    dtype=dtype)
dose_engine = DoseEngine(
    machine_config, 
    kernel_size,
    dose_grid_spacing=phantom.resolution,
    dose_grid_shape=phantom.density_image.shape,
    beam_template=beam_template,
    device=device,
    dtype=dtype,
    adjust_values=False
)
dose_engine.calibrate(110)
for field_size in FIELD_SIZES:
    number_of_beams = 1
    starting_angle = 0

    beam = Beam.create(
        gantry_angle_deg=0.0, 
        number_of_leaf_pairs=60, 
        collimator_angle_deg=0.0, 
        field_size_mm=(10 * field_size[0], 10 * field_size[1]), 
        iso_center=(200.0, 100.0, 200.0), 
        device=device, 
        dtype=dtype)
    beam.mu = 110 * beam.mu

    dose = dose_engine.compute_dose(
        beam,
        density_image=phantom.density_image).detach()
    print(dose_engine.iso_center_voxel)
    results.append(dose[0, *dose_engine.iso_center_voxel].item())

result_ratios = np.array(OF_MEASUREMENTS) / np.array(results)

In [ ]:
# Anisotropic geometry factors for FF visibility
k_X = D_X * (D_ISO - D_FF) / (D_ISO * (D_ISO - D_X))
k_Y = D_Y * (D_ISO - D_FF) / (D_ISO * (D_ISO - D_Y))

print(f"k_X = {k_X:.6f}, k_Y = {k_Y:.6f}")

# Measured Sc data
is_rect = [x != y for (x, y) in FIELD_SIZES]
is_sq = [x == y for (x, y) in FIELD_SIZES]
ax_sq = FIELD_SIZES[is_sq, 0]
ay_sq = FIELD_SIZES[is_sq, 1]
Sc_sq = result_ratios[is_sq]

# Rectangular fields
ax_rect = FIELD_SIZES[is_rect, 0]
ay_rect = FIELD_SIZES[is_rect, 1]
Sc_rect_meas = result_ratios[is_rect]

# Combine all fields
ax_all = FIELD_SIZES[:, 0]
ay_all = FIELD_SIZES[:, 1]
Sc_all = result_ratios

# Index of the 10x10 reference field
ref_idx = np.where((ax_all == 10.0) & (ay_all == 10.0))[0][0]

def f_frac_visibility_rect(ax, ay, sigma_FF):
    """
    Fraction of a 2D Gaussian (width sigma_FF) at the FF plane
    that is visible through the jaws for a rectangular field
    ax x ay at isocenter, given jaw depths and geometry.

    ax, ay: field side lengths at isocenter (cm)
    sigma_FF: Gaussian sigma at FF plane (cm)
    """
    hx = 0.5 * ax
    hy = 0.5 * ay

    # Half-widths of visible region at FF along X and Y
    wX = hx * k_X
    wY = hy * k_Y

    return erf(wX / (np.sqrt(2.0) * sigma_FF)) * \
           erf(wY / (np.sqrt(2.0) * sigma_FF))


def Sc_model_aniso_rect(p):
    """
    Headscatter Sc model with:
      - one isotropic Gaussian source at FF (sigma_FF)
      - anisotropic jaw-limited visibility (k_X, k_Y)
      - single scatter fraction S at 10x10.

    Parameters p:
      p[0] -> u (logit for scatter fraction S)
      p[1] -> logsigma_FF
    """
    u, logsigma = p

    # Scatter fraction at 10x10 (0..1)
    S = 1.0 / (1.0 + np.exp(-u))

    # Sigma at FF (>= 0.01 cm)
    sigma_FF = 0.01 + np.exp(logsigma)

    # Visibility fractions for all fields
    f_all = f_frac_visibility_rect(ax_all, ay_all, sigma_FF)
    f_ref = f_all[ref_idx]

    # Sc = primary + scatter, normalized so Sc_ref = 1
    Sc = (1.0 - S) + S * (f_all / f_ref)
    return Sc


def residuals_aniso_rect(p):
    return Sc_model_aniso_rect(p) - Sc_all


# Fit S and sigma_FF
S_guess = 0.055
sigma_guess = 1.9  # cm
p0 = np.array([
    np.log(S_guess / (1 - S_guess)),  # logit for S
    np.log(sigma_guess)               # logsigma
])

res_rect = least_squares(residuals_aniso_rect, p0)
u_fit_r, logsigma_fit_r = res_rect.x

S_fit = 1.0 / (1.0 + np.exp(-u_fit_r))
sigma_FF_fit = 0.01 + np.exp(logsigma_fit_r)

print("\n=== Refit including 40x5 and 5x40 ===")
print(f"Scatter fraction at 10x10 (S): {S_fit*100:.2f} %")
print(f"Primary fraction at 10x10:    {(1.0 - S_fit)*100:.2f} %")
print(f"Sigma_FF (cm):                {sigma_FF_fit:.4f}")

# Model Sc for all fields
Sc_fit_all = Sc_model_aniso_rect(res_rect.x)

print("\nField size      Measured Sc     Fitted Sc")
for ax, ay, sc_m, sc_f in zip(ax_all, ay_all, Sc_all, Sc_fit_all):
    print(f"{ax:4.1f}x{ay:4.1f}    {sc_m:0.9f}   {sc_f:0.9f}")

# Compute equivalent square sizes for plotting
def equivalent_square(ax, ay):
    return 2 * ax * ay / (ax + ay)

eq_sq_sq = equivalent_square(ax_sq, ay_sq)
eq_sq_rect = equivalent_square(ax_rect, ay_rect)

# Model curve for square fields from 2–40 cm
a_plot = np.linspace(2.0, 40.0, 200)
f_plot = f_frac_visibility_rect(a_plot, a_plot, sigma_FF_fit)
f_ref = f_frac_visibility_rect(10.0, 10.0, sigma_FF_fit)
Sc_plot_sq = (1.0 - S_fit) + S_fit * (f_plot / f_ref)

# Plot: model vs data
plt.figure()
plt.plot(a_plot, Sc_plot_sq, '-', label='Model (squares, 2–40 cm)')
plt.plot(eq_sq_sq, Sc_sq, 'o', label='Measured squares')

plt.plot(eq_sq_rect, Sc_rect_meas, 's', label='Measured rectangles')
plt.plot(eq_sq_rect, Sc_fit_all[-2:], 'x', label='Fitted rectangles')

plt.xlabel('Equivalent square size (cm)')
plt.ylabel('Headscatter factor Sc')
plt.title('Aniso-visibility FF model with rectangles')
plt.grid(True)
plt.legend()
plt.xlim(2, 40)
plt.tight_layout()
plt.show()

# Geometric penumbra sigmas at isocenter (X and Y)
def geom_penum_sigma(sigma_FF, d_iso, d_FF, d_jaw):
    """
    Geometric penumbra sigma at isocenter due to a finite
    scattering body at d_FF, limited by a jaw at d_jaw.
    """
    return sigma_FF * (d_iso - d_jaw) / (d_jaw - d_FF)

sigma_X_geom = geom_penum_sigma(sigma_FF_fit, D_ISO, D_FF, D_X)
sigma_Y_geom = geom_penum_sigma(sigma_FF_fit, D_ISO, D_FF, D_Y)

fwhm_x = 2.355 * sigma_X_geom
fwhm_y = 2.355 * sigma_Y_geom

print("\n=== Geometric penumbra from FF scatter body ===")
print(f"sigma_X_geom (crossline) = {sigma_X_geom:.4f} cm  (FWHM {fwhm_x:.4f} cm)")
print(f"sigma_Y_geom (inline)    = {sigma_Y_geom:.4f} cm  (FWHM {fwhm_y:.4f} cm)")

# These are the anisotropic sigmas to convolve the primary fluence with
sigma_x_conv = sigma_X_geom
sigma_y_conv = sigma_Y_geom

print("\n=== Convolution kernel sigmas at isocenter ===")
print(f"sigma_x_conv = {sigma_x_conv:.4f} cm")
print(f"sigma_y_conv = {sigma_y_conv:.4f} cm")



In [ ]:
kernel_mlc = precompute_head_scatter_kernel(sigma_x_conv, resolution_cm=0.1)
kernel_jaw = precompute_head_scatter_kernel(sigma_y_conv, resolution_cm=0.1)

kernel = np.outer(kernel_mlc, kernel_jaw)
print(kernel.shape)
corr_factor = np.max(kernel[kernel.shape[0]//2 - 50 : kernel.shape[0]//2 + 50, kernel.shape[1]//2 - 50 : kernel.shape[1]//2 + 50])

print("\n=== FINAL PARAMETERS FOR MACHINE CONFIG ===")
print(f'"head_scatter_amplitude": [{S_fit:.3f}, {corr_factor:.3f}],')
print(f'"head_scatter_sigma": [{sigma_x_conv:.2f}, {sigma_y_conv:.2f}]')